## Nous allons créer un Decision Tree permettant de classifier differents elements, en l'occurence des fruits en fonctions de trois attributs.

In [1]:
from __future__ import print_function
import numpy as np
import pandas as pd

In [2]:
training_data = pd.read_csv("data.csv")

### Visualisez les donnees (valeurs reelles) ###

In [3]:
training_data

,Color,Size,Season,Label
0,Red,1,Summer,Grape
1,Green,3,Spring,Apple
2,Red,3,Winter,Orange
3,Green,2,Winter,Kiwi
4,Yellow,3,Winter,Lemon
5,Green,2,Winter,Kiwi
6,Yellow,3,Winter,Lemon
7,Green,1,Summer,Grape
8,Red,3,Spring,Apple
9,Red,3,Winter,Orange


## Definissez X qui contiendra les attributs ET les labels (pour faciliter le partinioning par la suite)##

In [7]:
X = training_data.values

In [15]:
# Nous devons maintenant stocker les differents titres de categrories pour pouvoir afficher notre arbre
header = list(training_data.columns)
header

['Color', 'Size', 'Season', 'Label']

## Nous vous donnons la fonction permettant de trouver les differentes valeurs possibles pour chaque categorie, testez la pour mieux la comprendre ##

In [17]:
def unique_vals(rows, col):
    return set([row[col] for row in rows])

### Testez votre fonction pour les differents attributs et labels ###

In [18]:
# Afficher les differentes saisons avec unique_vals
unique_vals(X, 2)

{'Spring', 'Summer', 'Winter'}

In [19]:
# Afficher les differentes couleurs avec unique_vals
unique_vals(X, 0)

{'Green', 'Red', 'Yellow'}

In [20]:
# Afficher les differents fruits avec unique_vals
unique_vals(X, 3)

{'Apple', 'Grape', 'Kiwi', 'Lemon', 'Orange'}

### Definir la fonction permettant de compter le nombre de fruits présents *(astuce : créer un dictionnaire)* ###

In [283]:
def class_counts_cor(rows):
    counts = {}
    for row in rows:
        label = row[-1]
        if label not in counts:
            counts[label] = 0
        counts[label] += 1
    return counts

In [284]:
def class_counts(rows):
    counts = {}
    for row in rows :
        if row[-1] not in counts.keys() :
            counts[row[-1]] = 0
        counts[row[-1]] += 1
    return counts

In [285]:
## Lancez votre fonction pour votre dataset :
class_counts(X)
## Elle devrait retourner : {'Grape': 19, 'Apple': 19, 'Orange': 20, 'Kiwi': 22, 'Lemon': 20} 

{'Grape': 19, 'Apple': 19, 'Orange': 20, 'Kiwi': 22, 'Lemon': 20}

### Definition de la fonction is_numeric permettant de savoir si une valeur est de type int/float ou non. ###

In [286]:
def is_numeric(value):
    if type(value) == int or type(value) == float:
        return True
    else:
        return False

In [287]:
## Testez votre fonction pour une valeur numerique (int ou float) ...
is_numeric(9.8)

True

In [288]:
#... et pour une valeur non-numérique 
is_numeric("red")

False

### Pour partitionner le dataset, nous allons utiliser une classe "Questions" ! Cette classe enregistre un numero de colonne (Ex : 0 pour la couleur) et une valeur de la colonne (ex : Vert).  La fonction match est utilisee pour comparer la valeur d'un element a celle de la question. .###
#### Vous avez ici directement le constructeur et la fonction d'affichage. A vous de trouver la fonction match####

In [289]:
class Question:
    
    def __init__(self, column, value):
        # Le Constructeur  permet d'attribuer les attributs à la classe
        self.column = column
        self.value = value

    def match(self, example):
        # Compare l'attribut dans un exemple à l'attribut dans la question
        # ATTENTION : La valeur de retour doit etre un boolean (True / False)
        if is_numeric(self.value) :
            return example[self.column] >= self.value      
        else :
            return example[self.column] == self.value

    def __repr__(self):
        # This is just a helper method to print
        # the question in a readable format.
        condition = "=="
        if is_numeric(self.value):
            condition = ">="
        return "Is %s %s %s?" % (
            header[self.column], condition, str(self.value))

#### Testez vos questions

In [290]:
# Ecrivez une question pour un attribut numerique
q1 = Question(1,2)
q1

Is Size >= 2?

In [291]:
# Et une deuxieme pour un attribut catégorique : 
q2 = Question(0, "Red")
q2

Is Color == Red?

In [292]:
# Choisissons un example dans le dataset ...
example = X[5]

In [293]:
# ... Et testez si votre exemple est bien de couleur rouge  && size >= 3
q3 = Question(0, "Red")
q3.match(example)

False

### Grace a cette classe, nous allons pouvoir partitionner le dataset ###

In [294]:
def partition(rows, question):
    # Pour chaque ligne dans le dataset, partionnez le en 2 en fonction du resultat de la question.
    true_rows, false_rows = [], []
    for row in rows :
        if question.match(row) :
            true_rows.append(row)
        else :
            false_rows.append(row)
    return true_rows, false_rows

In [295]:
# Testons notre fonction partition pour isoler les lignes "rouges"
true_rows, false_rows = partition(X, Question(0, "Red"))

In [296]:
# Et verifiez le contenue de vos 2 nodes
true_rows

[array(['Red', 1, 'Summer', 'Grape'], dtype=object),
 array(['Red', 3, 'Winter', 'Orange'], dtype=object),
 array(['Red', 3, 'Spring', 'Apple'], dtype=object),
 array(['Red', 3, 'Winter', 'Orange'], dtype=object),
 array(['Red', 1, 'Summer', 'Grape'], dtype=object),
 array(['Red', 3, 'Winter', 'Orange'], dtype=object),
 array(['Red', 1, 'Summer', 'Grape'], dtype=object),
 array(['Red', 3, 'Winter', 'Orange'], dtype=object),
 array(['Red', 1, 'Summer', 'Grape'], dtype=object),
 array(['Red', 3, 'Winter', 'Orange'], dtype=object),
 array(['Red', 1, 'Summer', 'Grape'], dtype=object),
 array(['Red', 3, 'Winter', 'Orange'], dtype=object),
 array(['Red', 1, 'Summer', 'Grape'], dtype=object),
 array(['Red', 3, 'Winter', 'Orange'], dtype=object),
 array(['Red', 1, 'Summer', 'Grape'], dtype=object),
 array(['Red', 3, 'Winter', 'Orange'], dtype=object),
 array(['Red', 1, 'Summer', 'Grape'], dtype=object),
 array(['Red', 3, 'Winter', 'Orange'], dtype=object),
 array(['Red', 1, 'Summer', 'Grape'],

#### Maintenant un peu de maths !
#### Nous allons calculer l'impurity de chaque categorie (true / false) grace a GINI

In [297]:
def gini_cor(rows):
    counts = class_counts(rows)
    impurity = 1
    for lbl in counts:
        prob_of_lbl = counts[lbl] / float(len(rows))
        impurity -= prob_of_lbl**2
    return impurity

In [298]:
def gini(rows):
    impurity = 1
    cat = class_counts(rows)
    total = float(sum(cat.values()))
    for nb in cat.values() :
        impurity -= (float(nb)/total)**2
    return float(impurity)

In [299]:
# Pour verifier la fonction tester l'exemple suivant 
no_mixing = [['Apple'], ['Apple']]
# Que devrait-etre votre resultat ?
gini(no_mixing)

0.0

In [300]:
 # Maintenant avec un autre
some_mixing = [['Apple'], ['Orange'], ['Orange'], ['Apple']]
#  Que devriez vous obtenir comme resultat ?
gini(some_mixing)

0.5

In [301]:
# Et maintenant pour un dataset avec tous nos fruits
lots_of_mixing = [['Apple'],
                  ['Orange'],
                  ['Grape'],
                  ['Lemon'],
                  ['Kiwi']]
# Vous devriez obtenir environ 0.8
gini(lots_of_mixing)

0.7999999999999998

### Calculons l'information gain 

In [302]:
"""Information Gain :
The uncertainty of the starting node, minus the weighted impurity of
two child nodes.
"""

def info_gain(left, right, current_uncertainty):
    #left : true_rows
    #right : false_rows
    total_len = len(left) + len(right)
    info_gain = current_uncertainty - gini(left)*len(left)/total_len - gini(right)*len(right)/total_len
    return info_gain

In [316]:
def info_gain_cor(left, right, current_uncertainty):
    p = float(len(left)) / (len(left) + len(right))
    info_gain = current_uncertainty - p * gini(left) - (1 - p) * gini(right)
    return info_gain

In [317]:
# Calculez l'uncertainy de notre dataset.
current_uncertainty = gini(X)
current_uncertainty
# Vous devriez obtenir environ 0.8

0.7993999999999999

In [320]:
# Quel est notre information gain du Dataset si nous partitionnons avec la question couleur == rouge ??
true_rows, false_rows = partition(X, Question(0, "Red"))
info_gain(true_rows, false_rows, current_uncertainty)
# Resultat : environ 0.1835

0.1834941572089112

In [321]:
# De même pour la question "Size >= 2"
true_rows, false_rows = partition(X, Question(1, 2))
info_gain(true_rows, false_rows, current_uncertainty)
# Resultat : environ 0.1925

0.19248641975308622

### Pour ne pas tout tester manuellement, construisez la fonction vous permettant de trouver la question donnant le meilleur information gain en iterant sur chaque feature / value.

In [ ]:
def find_best_split(rows):
    best_gain = 0
    best_question = None  # 1 Objet Question
    current_uncertainty = gini(rows)
    n_features = len(rows[0]) - 1  # number of lines

    

    return best_gain, best_question

In [ ]:
### Testez votre fonction sur le dataset
best_gain, best_question = 
# Vous devriez avoir : is color == yellow ?

### Nous allons maintenant construire notre Decision Tree 
#### Les objets Leaf et Decision_Node sont donnés,  à vous de trouver la fonction qui permettra de construire l'arbre (build_tree)

In [ ]:
class Leaf:
    def __init__(self, rows):
        self.predictions = class_counts(rows)

In [ ]:
class Decision_Node:
    """A Decision Node asks a question.
    This holds a reference to the question, and to the two child nodes.
    """
    def __init__(self,
                 question,
                 true_branch,
                 false_branch):
        self.question = question
        self.true_branch = true_branch
        self.false_branch = false_branch

In [ ]:
def build_tree(rows):
    """Builds the tree.
    Rules of recursion: 1) Believe that it works. 2) Start by checking
    for the base case (no further information gain). 3) Prepare for
    giant stack traces.
    """
    # Rappelez-vous de la condition d'arret et de la creation d'une leaf


### Fonction d'affichage de l'arbre (c'est cadeau ;) )

In [ ]:
def print_tree(node, spacing=""):
    """World's most elegant tree printing function."""

    # Base case: we've reached a leaf
    if isinstance(node, Leaf):
        print (spacing + "Predict", node.predictions)
        return

    # Print the question at this node
    print (spacing + str(node.question))

    # Call this function recursively on the true branch
    print (spacing + '--> True:')
    print_tree(node.true_branch, spacing + "  ")

    # Call this function recursively on the false branch
    print (spacing + '--> False:')
    print_tree(node.false_branch, spacing + "  ")

### Maintenant, creez votre arbre et imprimez le ! ;)

In [ ]:
my_tree = build_tree(X)

In [ ]:
print_tree(my_tree)

### Construisez la fonction classify qui permettra de parcourir l'abre pour un exemple donné (et ainsi avoir nos resultats)

In [ ]:
def classify(row, node):


In [ ]:
# Test de la fonction avec le 16eme data sample. Vous devriez obtenir {'Kiwi': 22}
classify(X[15], my_tree)

In [ ]:
def print_leaf(counts):
    """A nicer way to print the predictions at a leaf."""
    total = sum(counts.values()) * 1.0
    probs = {}
    for lbl in counts.keys():
        probs[lbl] = str(int(counts[lbl] / total * 100)) + "%"
    return probs

In [ ]:
# Maintenant avec une fonction d'affichage un peu plus plaisante. Resultat : {'Grape': '100%'}
print_leaf(classify(X[0], my_tree))

In [ ]:
# Tester avec ce dataset, puis essayer de changer les valeurs pour mieux comprendre le comportement de l'arbre
testing_data = [
    ['Green', 2, 'Winter','Kiwi'],
    ['Yellow', 3, 'Winter', 'Lemon'],
    ['Red', 1, 'Summer', 'Grape'],
    ['Red', 2, 'Summer', 'Grape'],
    ['Yellow', 3, 'Winter', 'Lemon'],
]

In [ ]:
for row in testing_data:
    print ("Actual: %s. Predicted: %s" %
           (row[3], print_leaf(classify(row, my_tree))))